<a href="https://colab.research.google.com/github/dominikmeyer95/academic-output/blob/feature%2Fadvanced_derivatives/advanced_derivatives/retail_options_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Derivatives

* Group: Dominik Meyer, Sean Goedgeluk, Krzysztof Rentflejsz
* Assignment: Exploit retail investors' impact on option pricing 

# Key Ressources & Notes

**Tebaldi's opener:**
* Retail investors do not use options for hedging but for leveraging upside
* Thus, retail investors put price pressure on the call side of option markets
* This phenomenon becomes more severe the easier the access to derivatives becomes, e.g. via neobroker like Robinhood


**Notes on online articles:**
* https://qz.com/2043668/the-real-reason-options-trading-pushed-robinhood-shares-higher
* https://www.cboe.com/insights/posts/how-meme-stocks-impact-options-trading/

* In meme stocks, delta hedging does not work anymore, because the typical negative correlation of the assets in the hedging portfolio (options and the underlying stock) has turned. When a stock price goes up, its implied volatility typically goes down, since if a stock is going up, in theory it’s because investors have more certainty about its future cashflows and profits. Meme stocks flip this around, so that the equity may shoot up in price, but it’s reasonable to expect that this price increase is highly unstable, thus the implied volatility stays elevated. Investment banks hence hedge a short call by a long call position as this hedges the volatility risk as well.

**Notes on papers:**
* https://www.nhh.no/contentassets/2e16f7ca8332414a893cc34759004d60/retail-option_sep2022.pdf

* "*The effect of individual investors on implied volatility depends on the nature of their option trading. In an environment with imperfect market maker hedging and inventory risks, widespread uninformed purchasing of options  either calls or puts) will create upward pressure on prices that manifests as higher implied volatility. On the other hand, if retail investors tend to sell options, this will create downward pressure on prices and reduce implied volatility. In aggregate, retail investors are more likely to purchase options than write them, and net option purchases are strongest for short-dated out-of-the-money options.*"

* "*Our work contributes to several strands of literature. One area of research emphasizes the role of demand pressures on option markets. For example, Bollen and Whaley (2004) argue that buying pressure helps explain differences in the shape of the moneyness curves between index and stock options. Garleanu, Pedersen, and Poteshman (2009) models demand-pressure effects and finds that proxies for option demand are related to the moneyness smirk. [...] Our evidence documents the important role that speculative retail investor demand pressure can have on the implied volatility surface, which suggests caution is warranted when interpreting the implied volatility surface as reflecting information about underlying firm fundamentals.*"

* "*Our evidence documents the important role that speculative retail investor
demand pressure can have on the implied volatility surface, which suggests caution is warranted when interpreting the implied volatility surface as reflecting information about underlying firm fundamentals.*"

* https://www.math.kth.se/matstat/seminarier/reports/M-exjobb14/140909.pdf

* "*The SVI implied volatility model is a parametric model for stochastic implied volatility. The SVI is interesting because of the possibility to state explicit conditions on its parameters so that the model does not generate prices where static arbitrage opportunities can occur. Calibration of the SVI model to real market data requires non-linear optimization algorithms and can be quite time consuming.*"


**Ideas:**
* Have a look at implied volatility and its variation
* Have a look at the stock price and its variation
* Have a look at the rolling correlation between both
* (Have a look at the volatility surface as it might be less skewed based on that information) 
  * Not possible, since we just have the atm strike in WRDS OptionMetrics
* Have a look at the delta hedging differences?

# Table of Contents

In [2]:
# import required packages
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
# import and format spx option data
spx_opt = pd.read_csv('https://raw.githubusercontent.com/dominikmeyer95/academic-output/feature/advanced_derivatives/advanced_derivatives/option_data_spx.csv')
spx_opt['date'] = [datetime.strptime(str(date),'%Y%m%d') for date in spx_opt['date']]
spx_opt['impl_volatility'] = spx_opt['impl_volatility']*100
spx_opt = spx_opt[(spx_opt['days'] == 30) & (spx_opt['cp_flag'] == 'C')]
# import and format spx stock data
spx_sto = pd.read_csv('https://raw.githubusercontent.com/dominikmeyer95/academic-output/feature/advanced_derivatives/advanced_derivatives/stock_data_spx.csv')
spx_sto['date'] = [datetime.strptime(str(date),'%Y%m%d') for date in spx_sto['date']]
# merge and format datasets
spx = pd.merge(spx_opt, spx_sto, on=['date'], how='left', suffixes=('', '_drop'))
spx.drop([col for col in spx.columns if 'drop' in col], axis=1, inplace=True)
# compute rolling correlation between implied volatility and index level
spx['rolling_correlation'] = spx['impl_volatility'].rolling(30).corr(spx['close'])
spx

,secid,date,days,strike_price,premium,impl_volatility,cp_flag,ticker,index_flag,close,cfadj,shrout,rolling_correlation
0,108105,2010-12-31,30,1255.787086,23.327599,16.2468,C,SPX,1,1257.64,1,0,NaN
1,108105,2011-01-03,30,1270.027164,23.155669,15.9462,C,SPX,1,1271.89,1,0,NaN
2,108105,2011-01-04,30,1268.338072,22.577426,15.5686,C,SPX,1,1270.20,1,0,NaN
3,108105,2011-01-05,30,1274.693239,22.588101,15.4984,C,SPX,1,1276.56,1,0,NaN
4,108105,2011-01-06,30,1271.988473,23.356861,16.0600,C,SPX,1,1273.85,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765,108105,2021-12-27,30,4788.494417,73.023901,13.3404,C,SPX,1,4791.19,1,0,-0.921903
2766,108105,2021-12-28,30,4783.621575,72.142463,13.1927,C,SPX,1,4786.35,1,0,-0.924695
2767,108105,2021-12-29,30,4790.266566,72.133035,13.1726,C,SPX,1,4793.06,1,0,-0.931484
2768,108105,2021-12-30,30,4775.958251,72.901746,13.3528,C,SPX,1,4778.73,1,0,-0.937364


In [4]:
# import and format tsla option data
tsla_opt = pd.read_csv('https://raw.githubusercontent.com/dominikmeyer95/academic-output/feature/advanced_derivatives/advanced_derivatives/option_data_tsla.csv')
tsla_opt['date'] = [datetime.strptime(str(date),'%Y%m%d') for date in tsla_opt['date']]
tsla_opt['impl_volatility'] = tsla_opt['impl_volatility']*100
tsla_opt = tsla_opt[(tsla_opt['days'] == 30) & (tsla_opt['cp_flag'] == 'C')]
# import and format tsla stock data
tsla_sto = pd.read_csv('https://raw.githubusercontent.com/dominikmeyer95/academic-output/feature/advanced_derivatives/advanced_derivatives/stock_data_tsla.csv')
tsla_sto['date'] = [datetime.strptime(str(date),'%Y%m%d') for date in tsla_sto['date']]
tsla_sto['close'][tsla_sto['cfadj'] == 1] = tsla_sto['close'][tsla_sto['cfadj'] == 1] / 5
# merge and format datasets
tsla = pd.merge(tsla_opt, tsla_sto, on=['date'], how='left', suffixes=('', '_drop'))
tsla.drop([col for col in tsla.columns if 'drop' in col], axis=1, inplace=True)
# compute rolling correlation between implied volatility and stock price
tsla['rolling_correlation'] = tsla['impl_volatility'].rolling(30).corr(spx['close'])
tsla

<ipython-input-4-bc22d5b32ea7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsla_sto['close'][tsla_sto['cfadj'] == 1] = tsla_sto['close'][tsla_sto['cfadj'] == 1] / 5


,secid,date,days,strike_price,premium,impl_volatility,cp_flag,ticker,index_flag,close,cfadj,shrout,rolling_correlation
0,143439,2010-12-31,30,26.635991,1.727454,56.7751,C,TSLA,0,5.326,1,93271,NaN
1,143439,2011-01-03,30,26.626077,1.792789,58.9496,C,TSLA,0,5.324,1,93271,NaN
2,143439,2011-01-04,30,26.676120,1.758847,57.7223,C,TSLA,0,5.334,1,93271,NaN
3,143439,2011-01-05,30,26.836192,1.744676,56.9140,C,TSLA,0,5.366,1,93271,NaN
4,143439,2011-01-06,30,27.886495,1.796009,56.3809,C,TSLA,0,5.576,1,93271,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765,143439,2021-12-27,30,1094.443894,79.493497,63.6184,C,TSLA,0,1093.940,5,1004260,-0.646182
2766,143439,2021-12-28,30,1088.966259,78.114524,62.8268,C,TSLA,0,1088.470,5,1004260,-0.638249
2767,143439,2021-12-29,30,1086.675048,77.617082,62.5571,C,TSLA,0,1086.190,5,1004260,-0.636142
2768,143439,2021-12-30,30,1070.815708,74.968875,61.3142,C,TSLA,0,1070.340,5,1004260,-0.646828


In [6]:
# initiate multiplot
fig = make_subplots(rows=2, cols=2, vertical_spacing=0.15, horizontal_spacing=0.20,
specs=[[{"secondary_y": True}, {"secondary_y": True}],[{"secondary_y": False}, {"secondary_y": False}]],
subplot_titles=('SPX Implied Volatility vs. Index Level', 'TSLA Implied Volatility vs. Stock Price', 'SPX 30-Days Rolling Correlation', 'TSLA 30-Days Rolling Correlation'))
# plot implied volatility vs. price and its rolling correlation
fig.add_trace(go.Scatter(x=spx['date'], y=spx['impl_volatility'], name='implied volatility', marker_color='slateblue', legendgroup='spx', legendgrouptitle_text='spx'), row=1, col=1, secondary_y=False)
fig.add_trace(go.Scatter(x=spx['date'], y=spx['close'], name='index level', marker_color='#bcbd22', legendgroup='spx'), row=1, col=1, secondary_y=True)
fig.add_trace(go.Scatter(x=spx['date'], y=spx['rolling_correlation'], name='rolling correlation', marker_color='tomato', legendgroup='spx'), row=2, col=1)
fig.add_trace(go.Scatter(x=tsla['date'], y=tsla['impl_volatility'], name='implied volatility', marker_color='slateblue', legendgroup='tsla', legendgrouptitle_text='tsla'), row=1, col=2, secondary_y=False)
fig.add_trace(go.Scatter(x=tsla['date'], y=tsla['close'], name='stock price', marker_color='#bcbd22', legendgroup='tsla'), row=1, col=2, secondary_y=True)
fig.add_trace(go.Scatter(x=tsla['date'], y=tsla['rolling_correlation'], name='rolling correlation', marker_color='tomato', legendgroup='tsla'), row=2, col=2)
# format and display multiplot
fig.update_layout(template="plotly_dark", height=800, title_x=0.5, showlegend=False, legend_tracegroupgap=25, legend_groupclick="toggleitem", legend=dict(yanchor="top", y=0.65))
fig.update_xaxes(title_text='date', showgrid=True, gridcolor="grey", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey")
fig.update_yaxes(showgrid=True, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=False)
fig.update_yaxes(showgrid=False, gridcolor="grey", ticks="outside", tickwidth=1, tickcolor='grey', ticklen=5, linecolor="grey", secondary_y=True)
fig['layout']['yaxis1']['title']='implied volatility in %'
fig['layout']['yaxis2']['title']='index level in usd'
fig['layout']['yaxis3']['title']='implied volatility in %'
fig['layout']['yaxis4']['title']='stock price in usd'
fig['layout']['yaxis5']['title']='correlation'
fig['layout']['yaxis5']['range']=[-1,1.1]
fig['layout']['yaxis6']['title']='correlation'
fig['layout']['yaxis6']['range']=[-1,1.1]
fig.show()

In [ ]:
# create implied volatility surface subplots (temp dummy)

# read data from a csv
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')

# initialize figure with 2 subplots
fig = make_subplots(rows=1, cols=2, specs= [[{'is_3d': True}, {'is_3d': True}]])
fig.add_trace(go.Surface(z=z_data.values, showscale=False), row=1, col=1)
fig.add_trace(go.Surface(z=z_data.values, showscale=False), row=1, col=2)
fig.update_layout(template="plotly_dark", title_text='Implied Volatility Surface')
fig.show()